# *可选* 将逻辑与小部件分离

设计图形用户界面的一个关键原则是将应用程序的逻辑与用户看到的图形小部件分离。例如，在这个超级简单的密码生成器小部件中，基本的逻辑是根据给定的长度构造一个随机字母序列。我们将把这个逻辑隔离到一个没有任何小部件的函数中。这个函数接受一个密码长度并返回生成的密码字符串。

In [ ]:
import ipywidgets as widgets

In [ ]:
def calculate_password(length):
    import string
    import secrets
    
    # Gaenerate a list of random letters of the correct length.
    password = ''.join(secrets.choice(string.ascii_letters) for _ in range(length))

    return password

在下面的单元格中测试该函数几次，使用不同的长度。请注意，与我们第一次实现时不同，你可以在不定义任何小部件的情况下测试这个函数。这意味着你可以只为逻辑编写测试，作为库的一部分使用该函数等等。

In [ ]:
calculate_password(10)

## 图形控件

构建图形用户界面小部件的代码与之前的版本相同。

In [ ]:
helpful_title = widgets.HTML('Generated password is:')
password_text = widgets.HTML('No password yet')
password_text.layout.margin = '0 0 0 20px'
password_length = widgets.IntSlider(description='Length of password',
                                   min=8, max=20,
                                   style={'description_width': 'initial'})

password_widget = widgets.VBox(children=[helpful_title, password_text, password_length])
password_widget


## 连接逻辑到小部件

当滑块 `password_length` 变化时，我们希望调用 `calculate_password` 来生成一个新密码，并将小部件 `password` 的值设置为函数调用的返回值。

`update_password` 以 `password_length` 的变化为参数，并用 `calculate_password` 的结果更新 `password_text`。

In [ ]:
def update_password(change):
    length = int(change.new)
    new_password = calculate_password(length)
    
    # NOTE THE LINE BELOW: it relies on the password widget already being defined.
    password_text.value = new_password
    
password_length.observe(update_password, names='value')

现在连接已经建立，试着移动滑块，你应该能看到密码更新。

In [ ]:
password_widget

## 分离关注点的好处

这种方法的优点包括：

+ `ipywidgets` 中的变化只会影响你的控件设置。
+ 功能逻辑的变化只会影响你的密码生成函数。如果你决定单纯的字母密码不够安全，并且决定加入一些数字和/或特殊字符，唯一需要修改的代码就是 `calculate_password` 函数中的部分。
+ 你可以为 `calculate_password` 函数编写单元测试 —— 这是执行重要工作的地方 —— 而无需在浏览器中测试图形控件。

## 使用 interact

请注意，使用 `interact` 构建这个图形界面也强调了逻辑和控件之间的分离。然而，`interact` 对控件的布局有更强的规定性：控件位于函数输出的上方，采用垂直布局 (vbox)。对于快速构建初始图形界面，这通常非常方便，但对于更复杂的图形界面来说，这种布局可能会显得限制性较强。

In [ ]:
from ipywidgets import interact
from IPython.display import display
interact(calculate_password, length=(8, 20));

我们可以通过打印结果，而不仅仅是返回字符串，使得 `interact` 显得更加美观。这次我们使用 `interact` 作为装饰器。

In [ ]:
@interact(length=(8, 20))
def print_password(length):
    print(calculate_password(length))